# Current

## Create test example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import string, random, toolz, pandas, importlib, sys, os

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    191557.000000
mean        260.018919
std         258.235286
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3084.000000
dtype: float64

## Run current version

In [5]:
!git checkout main
#importlib.reload(dimod)

Switched to branch 'main'
Your branch and 'upstream/main' have diverged,
and have 1 and 2 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)


In [6]:
from dimod import BinaryPolynomial as BPold

In [7]:
BPold.__init__??

Signature: BPold.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        self._terms = terms = {}
        for term, bias in poly:

            fsterm = asfrozenset(term)

            # when SPIN-valued, s^2 == 1, so we need to handle that case
            # in BINARY, x^2 == x
            if len(fsterm) < len(term) and vartype is Vartype.SPIN:
                new = set()
                term = tuple(term)  # make sure it has .count
                for v in fsterm:
                    if term.count(v) % 2:
                        new.add(v)
                fsterm = frozenset(new)

            if fsterm in terms:
                terms[fsterm] += bias
            else:
                terms[fsterm] = bias

        self.vartype = 

In [8]:
%prun -s cumtime -l 20 BPold(terms, 'SPIN')

         9687489 function calls (9687409 primitive calls) in 28.809 seconds

   Ordered by: cumulative time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   28.809   28.809 {built-in method builtins.exec}
        1    0.065    0.065   28.809   28.809 <string>:1(<module>)
        1    0.000    0.000   28.743   28.743 decorators.py:289(new_f)
        1    2.587    2.587   28.743   28.743 polynomial.py:89(__init__)
  5468426   25.143    0.000   25.143    0.000 {method 'count' of 'tuple' objects}
   191557    0.649    0.000    0.677    0.000 polynomial.py:32(asfrozenset)
  3452632    0.297    0.000    0.297    0.000 {method 'add' of 'set' objects}
   383116    0.039    0.000    0.039    0.000 {built-in method builtins.len}
   191567    0.028    0.000    0.028    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
    

# Exit interpreter
I just can'get module reloading to work properly here ...

In [9]:
## 
exit()
#del BPold

# Proposed

## Create test example

In [1]:
import string, random, toolz, pandas, importlib, sys, os

In [2]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [3]:
pandas.Series(len(term) for term, _ in terms).describe()

count    192902.000000
mean        258.198977
std         257.967508
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3254.000000
dtype: float64

## Run proposed version

In [4]:
!git checkout binary-polynomial
!touch -c ./dimod/higherorder/polynomial.py
#importlib.reload(dimod)

Already on 'binary-polynomial'
Your branch is up to date with 'origin/binary-polynomial'.


In [5]:
from dimod import BinaryPolynomial as BPnew

In [6]:
BPnew.__init__??

Signature: BPnew.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms

        self._terms = terms = defaultdict(int)
        for term, bias in poly:
            terms[freeze_term(term, vartype)] += bias
        
        self.vartype = vartype
File:      ~/dimod/dimod/higherorder/polynomial.py
Type:      function


In [7]:
%prun -s cumtime -l 20  BPnew(terms, 'SPIN')

         578909 function calls (578829 primitive calls) in 4.456 seconds

   Ordered by: cumulative time
   List reduced from 45 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.456    4.456 {built-in method builtins.exec}
        1    0.073    0.073    4.456    4.456 <string>:1(<module>)
        1    0.000    0.000    4.383    4.383 decorators.py:289(new_f)
        1    0.188    0.188    4.383    4.383 polynomial.py:111(__init__)
   192902    4.167    0.000    4.195    0.000 polynomial.py:43(freeze_term)
   192912    0.015    0.000    0.015    0.000 {built-in method builtins.isinstance}
   192903    0.013    0.000    0.013    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}
     41/1    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__